In [1]:
using JuMP
using Cbc: CbcSolver
using Clp: ClpSolver
import JSON

In [2]:
const res = open("data0.json", "r") do f
    data = readstring(f)
    JSON.Parser.parse(data)
end

const maxwidth = res["maxwidth"]
const cost = res["cost"]
const prices = Float64.(res["prices"])
const widths = Float64.(res["widths"])
const demand = Float64.(res["demand"])
const nwidths = length(prices);

In [3]:
"""
    subproblem tries to find the best feasible pattern 
    maximizing reduced cost and respecting max roll width
    corresponding to a multiple-item knapsack
"""
function subproblem(reduced_costs, sizes, maxcapacity)
    subm = Model(solver = CbcSolver())
    n = length(reduced_costs)
    xs = @variable(subm, xs[1:n] >= 0, Int)
    @constraint(subm, sum(xs.*sizes) <= maxcapacity)
    @objective(subm, Max, sum(xs.*reduced_costs))
    solve(subm)
    return round.(Int,getvalue(xs)), round(Int,getobjectivevalue(subm))
end

subproblem

In [4]:
function init_master(maxwidth, widths, rollcost, demand, prices)
    n = length(widths)
    ncols = length(widths)
    patterns = spzeros(UInt16,n,ncols)
    for i in 1:n
        patterns[i,i] = min(floor(Int,maxwidth/widths[i]),round(Int,demand[i]))
    end
    m = Model(solver = ClpSolver())
    θ = @variable(m, θ[1:ncols] >= 0)
    @objective(m, Min,
        sum(θ[p]*(rollcost - sum(patterns[j,p]*prices[j] for j=1:n)) for p in 1:ncols)
    )
    @constraint(m, demand_satisfaction[j=1:n], sum(patterns[j,p]*θ[p] for p in 1:ncols)>=demand[j])
    if solve(m) != :Optimal
        warn("No optimal")
    end
    return (m, getvalue(θ), demand_satisfaction, patterns)
end

init_master (generic function with 1 method)

In [5]:
(m, θ, demand_satisfaction, patterns) = init_master(maxwidth, widths, cost, demand, prices);

In [6]:
reduced_costs = getdual(demand_satisfaction)+prices;

In [7]:
newcol, newobj = subproblem(reduced_costs, widths, maxwidth)

([2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0], 1193)

In [8]:
netcost = (cost-sum(newcol[j]*(getdual(demand_satisfaction[j])+prices[j]) for j in 1:nwidths))

-443.18181818181824

In [9]:
function column_generation(maxwidth, widths, rollcost, demand, prices; maxcols = 5000)
    (m, θ, demand_satisfaction, patterns) = init_master(maxwidth, widths, rollcost, demand, prices)
    ncols = nwidths
    while ncols <= maxcols
        reduced_costs = getdual(demand_satisfaction) + prices
        newcol, newobj = subproblem(reduced_costs, widths, maxwidth)
        netcost = cost - sum(newcol[j]*(getdual(demand_satisfaction)[j]+prices[j]) for j in 1:nwidths)
        println("New reduced cost: $netcost")
        if netcost >= 0
            return (:Optimal, patterns, getvalue(θ))
        end
        patterns = hcat(patterns, newcol)
        ncols += 1
        m = Model(solver = ClpSolver())
        θ = @variable(m, θ[1:ncols] >= 0)
        @objective(m, Min,
            sum(θ[p]*(rollcost - sum(patterns[j,p]*prices[j] for j=1:nwidths)) for p in 1:ncols)
        )
        @constraint(m, demand_satisfaction[j=1:nwidths], sum(patterns[j,p]*θ[p] for p in 1:ncols)>=demand[j])
        if solve(m) != :Optimal
            warn("No optimal")
            return (status(m), patterns, getvalue(θ))
        end
    end
    return (:NotFound, patterns, :NoVariable)
end

column_generation (generic function with 1 method)

In [10]:
status, patterns, θ = column_generation(maxwidth, widths, cost, demand, prices, maxcols = 500);
status

New reduced cost: -443.18181818181824
New reduced cost: -375.0
New reduced cost: -264.0
New reduced cost: -250.0
New reduced cost: -187.5
New reduced cost: -150.0
New reduced cost: -150.0
New reduced cost: -107.14285714285711
New reduced cost: -97.5
New reduced cost: -107.14285714285734
New reduced cost: -72.0
New reduced cost: -53.571428571428555
New reduced cost: -53.125
New reduced cost: -50.0
New reduced cost: -43.40625
New reduced cost: -36.0
New reduced cost: -34.625
New reduced cost: -41.5
New reduced cost: -21.8515625
New reduced cost: -22.159090909090878
New reduced cost: -20.625
New reduced cost: -16.304347826086314
New reduced cost: -16.304347826086996
New reduced cost: -20.310344827586277
New reduced cost: -18.0
New reduced cost: -8.837209302325732
New reduced cost: -6.060606060606119
New reduced cost: 0.0


:Optimal

In [18]:
# take worse case from linear solution, round up 
intial_integer = ceil.(Int,θ);
println(sum(θ))

446.1000000000001


In [20]:
"""
    From patterns built in the column generation phase, find an integer solution
"""function branched_model(patterns, demand, rollcost, prices; npatts = size(patterns)[2], initial_point = zeros(Int,npatts))
    npatts = size(patterns)[2]
    m = Model(solver = CbcSolver())
    θ = @variable(m, θ[p = 1:npatts] >= 0, Int, start = initial_point[p])
    @objective(m, Min,
        sum(θ[p]*(rollcost - sum(patterns[j,p]*prices[j] for j=1:nwidths)) for p in 1:npatts)
    )
    @constraint(m, demand_satisfaction[j=1:nwidths], sum(θ[p]*patterns[j,p] for p in 1:npatts) >= demand[j])
    status = solve(m)
    return (status, round.(Int,(getvalue(θ))))
end

branched_model

In [21]:
status, θ_final = branched_model(patterns, demand, cost, prices; initial_point = intial_integer)

Presolve 0 (-16) rows, 0 (-43) columns and 0 (-83) elements
Optimal - objective value 154188
After Postsolve, objective 154188, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 154188 - 0 iterations time 0.002, Presolve 0.00
Cbc0045I Solution with objective value 154188 saved


(:Optimal, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  58, 32, 0, 2, 44, 43, 50, 19, 5, 6])

In [ ]:
Int.(patterns)

In [ ]:
# final excess rolls
patterns * θ_final - demand

In [22]:
sum(θ_final)

447